<a href="https://colab.research.google.com/github/verma-saloni/Articles/blob/master/Fasttext1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install unidecode
!pip install word2number
!pip install contractions

  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=beff3aea39d80836f467cac9c7d0b1a0d2525996dd51696290e490707b2583f5
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
     |████████████████████████████████| 284 kB 8.1 MB/s 
     |████████████████████████████████| 321 kB 66.8 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=ac55d29d01d703d087d88315a9bf7a8bafba6aa7f47f095315e1fe5af53cd6e6
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [6]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1
df= df_fake.head(50).append(df_real.head(50))
df.reset_index(drop=True, inplace=True)

In [74]:
print(df.columns)

Index(['id', 'news_url', 'title', 'tweet_ids', 'labelML'], dtype='object')


In [75]:
#print(df)

In [76]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)

In [77]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [78]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
#---------df1.iloc[:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__' + x)
#print(df1)

                                                title  labelML
0   did miley cyrus and liam hemsworth secretly ge...        0
1   paris jackson cara delevingne enjoy night out ...        0
2   celebrities join tax march in protest of donal...        0
3   cindy crawford daughter kaia gerber wears wig ...        0
4              full list of oscar nominations variety        0
..                                                ...      ...
95  michael buble and wife luisana lopilato welcom...        1
96      selena gomez is going to keep her blonde hair        1
97            netflix new releases coming in february        1
98  halsey calls iggy azalea king moron and calls ...        1
99  delilah opens up about son suicide and heartbr...        1

[100 rows x 2 columns]



 #df = df.loc[:,"title"]
 # just for loc practice, ignore.

 X_train - This includes your all independent variables,these will be used to train the model, also as we have specified the test_size = 0.4, this means 60% of observations from your complete data will be used to train/fit the model and rest 40% will be used to test the model.

2). X_test - This is remaining 40% portion of the independent variables from the data which will not be used in the training phase and will be used to make predictions to test the accuracy of the model.

3). y_train - This is your dependent variable which needs to be predicted by this model, this includes category labels against your independent variables, we need to specify our dependent variable while training/fitting the model.

4). y_test - This data has category labels for your test data, these labels will be used to test the accuracy between actual and predicted categories.


In [79]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
df1.iloc[:49, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__0 ' + x)
df1.iloc[50:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__1 ' + x)
print(df1)

                                                title  labelML
0                                                 NaN        0
1   __label__0 paris jackson cara delevingne enjoy...        0
2   __label__0 celebrities join tax march in prote...        0
3   __label__0 cindy crawford daughter kaia gerber...        0
4   __label__0 full list of oscar nominations variety        0
..                                                ...      ...
95  __label__1 michael buble and wife luisana lopi...        1
96  __label__1 selena gomez is going to keep her b...        1
97  __label__1 netflix new releases coming in febr...        1
98  __label__1 halsey calls iggy azalea king moron...        1
99  __label__1 delilah opens up about son suicide ...        1

[100 rows x 2 columns]


In [71]:
df1.reset_index(drop=True, inplace=True)
X, y = df1.iloc[:, 1:], df1['labelML']

# Create the training and test sets #IMP: use stratify when dataset is not balanced X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)
#https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 1.000000


In [72]:
df2=df1
df2 = df2.reindex(columns=['labelML', 'title'])
#print(df1)

In [80]:
train, test = train_test_split(df2, test_size=0.2)

In [82]:
# Saving the CSV file as a text file to train/test the classifier
import csv #needed
train[['labelML', 'title']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

test[['labelML', 'title']].to_csv('test.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [92]:
!pip install fasttext
import fasttext
# Training the fastText classifier
model = fasttext.train_supervised('train.txt', wordNgrams = 2)

# Evaluating performance on the entire test file
model.test('test.txt')                      

# Predicting on a single input
test['labelML'] = test['labelML'].astype(str)
model.predict(test.iloc[6, 0])

# Save the trained model
#model.save_model('model.bin')

(('__label__1',), array([0.50024968]))

In [90]:
print(y_test)

8     0
70    1
82    1
28    0
63    1
0     0
5     0
50    1
81    1
4     0
Name: labelML, dtype: int64
